## Combine the train and test data and split them into train and test data


In [ ]:
%cd ..
%ls

In [1]:
dataset_name = 'madasr23dataset'
origin_train_file_path = 'corpus/bn/train/text'
origin_dev_file_path = 'corpus/bn/dev/text'

In [2]:
import pandas as pd

# Read the data
train_data = pd.read_csv(origin_train_file_path, sep='\t', header=None)
dev_data = pd.read_csv(origin_dev_file_path, sep='\t', header=None)

# Combine the train and test data
data = pd.concat([train_data, dev_data], ignore_index=True)

# The data looks like this:
# sid_txtid_uttid text
# 16777288_629046_281474981563595 বাঁশের প্রায়


# Convert the data to the format of sid txtid uttid text, where each value in own column
data = data[0].str.split('_', n=2, expand=True) 
data = pd.concat([data[0], data[1], data[2].str.split(' ', n=1, expand=True)], axis=1)
data.columns = ['spkid', 'txtid', 'uttid', 'text']

print(data.head())

      spkid   txtid            uttid  \
0  16777288  629046  281474981563595   
1  16777288  629072  281474981405386   
2  16777288  629108  281474981563619   
3  16777288  629112  281474981581650   
4  16777288  629121  281474981582236   

                                                text  
0  বাঁশের প্রায় দশ হাজার প্রজাতি হয় যেমন বাম্বুসা...  
1  এক ধরনের পদ্ধতি যেটা দিয়ে শস্য থেকে খোসা ছাড়ান...  
2         গুগলি বা ঝিনুকের মানে হল ওয়েস্টার আমরা খাই  
3            বাগ আরশোলা ইত্যাদি সব পোকা গুলোর চাষ হয়  
4        প্রাকৃতিক সব জিনিস গুলো দিয়া যখন চাষ করা হয়  


In [3]:
# Convert the text to phonemes
from phonemizer import phonemize
phonemes = phonemize(data['text'], backend='espeak', language='bn', strip=True, preserve_punctuation=True, with_stress=True, njobs=8)
phonemes = pd.DataFrame(phonemes)
phonemes.columns = ['phonemes']

data = pd.concat([data, phonemes], axis=1)
print(data.head())

      spkid   txtid            uttid  \
0  16777288  629046  281474981563595   
1  16777288  629072  281474981405386   
2  16777288  629108  281474981563619   
3  16777288  629112  281474981581650   
4  16777288  629121  281474981582236   

                                                text  \
0  বাঁশের প্রায় দশ হাজার প্রজাতি হয় যেমন বাম্বুসা...   
1  এক ধরনের পদ্ধতি যেটা দিয়ে শস্য থেকে খোসা ছাড়ান...   
2         গুগলি বা ঝিনুকের মানে হল ওয়েস্টার আমরা খাই   
3            বাগ আরশোলা ইত্যাদি সব পোকা গুলোর চাষ হয়   
4        প্রাকৃতিক সব জিনিস গুলো দিয়া যখন চাষ করা হয়   

                                            phonemes  
0  bˈãʃeɾ pɾˈajo dˈɔʃ hˈaɟaɾ pɾˈoɟatˌi hˈɔjo dʒˈ...  
1  ˈek dʰˈɔɾɔnˌeɾ pˈɔddʰɔtˌi dʒˈeʈa dˈie ʃˈɔssɔ t...  
2  ɡˈuɡɔlˌi bˈa ɟʰˈinukˌeɾ mˈane hˈɔl ˈoːeʃʈˌaɾ ˌ...  
3  bˈaɡ ˈaɾɔʃˌola ˈitædˌi ʃˈɔb pˈoka ɡˈuloɾ tʃˈaʃ...  
4  pɾakɾˈitik ʃˈɔb ɟˈiniʃ ɡˈulo dˈia dʒˈɔkʰɔn tʃˈ...  


### Save the data to a csv file

In [4]:
data.to_csv(f'filelists/{dataset_name}.csv', index=False)